In [89]:
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
from huggingface_hub import InferenceApi, InferenceClient
from hugchat import hugchat
import os, json
from langchain_community.document_loaders import TextLoader, WebBaseLoader, OnlinePDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from typing import List, Dict, Any, Tuple, Optional, Literal, Annotated
from fastapi import FastAPI
from pydantic import BaseModel, Field, conint, confloat
from enum import Enum
from pydantic import BaseModel, Field, conint, confloat, constr, StringConstraints

In [68]:
pip install fastapi uvicorn pydantic

In [26]:
!pip install huggingface_hub
!pip install hugchat

In [6]:
!pip -q install huggingface_hub

In [7]:
!pip install -U langchain langchain-community langchain-huggingface
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 128.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.6 MB/s

In [15]:
!apt -q install -y poppler-utils tesseract-ocr
!pip -q install "unstructured[pdf]" unstructured-inference "pillow>=9.5.0" pdfminer.six

Reading package lists...
Building dependency tree...
Reading state information...
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.11 [186 kB]
Fetched 186 kB in 0s (425 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.11_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.11) ...
Setting up poppler-utils (22.02.0-2ubuntu0.11) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━

In [103]:
urls = [
"https://www.cms.gov/medical-bill-rights/help/guides/explanation-of-benefits",
"https://www.bluecrossnc.com/members/knowledge-center/common-health-insurance-terms",
"https://www.bluecrossnc.com/members/knowledge-center/how-to-read-eob",
"https://www.cigna.com/knowledge-center/explanation-of-benefits",
"https://healthy.kaiserpermanente.org/maryland-virginia-washington-dc/learn/understand-health-plan-documents",
"https://www.adp.com/resources/articles-and-insights/articles/a/aca-compliance.aspx#:~:text=Each%20year%2C%20employers%20are%20required,C%20forms%20that%20were%20issued.",
"https://www.federalregister.gov/documents/2014/03/10/2014-05051/information-reporting-of-minimum-essential-coverage",
"https://www.irs.gov/affordable-care-act/questions-and-answers-on-information-reporting-by-health-coverage-providers-section-6055",
"https://www.irs.gov/affordable-care-act/employers/questions-and-answers-on-reporting-of-offers-of-health-insurance-coverage-by-employers-section-6056",
"https://www.irs.gov/affordable-care-act/employers/information-reporting-by-applicable-large-employers",
"https://www.irs.gov/retirement-plans/plan-participant-employee/401k-resource-guide-plan-participants-401k-plan-overview",
"https://www.irs.gov/retirement-plans/plan-participant-employee/retirement-topics-401k-and-profit-sharing-plan-contribution-limits",
"https://www.irs.gov/retirement-plans/plan-participant-employee/401k-resource-guide-plan-participants-general-distribution-rules",
"https://www.irs.gov/retirement-plans/plan-participant-employee/401k-resource-guide-plan-participants-summary-plan-description",
"https://www.irs.gov/retirement-plans/plan-participant-employee/retirement-plan-notices-to-interested-parties",
"https://www.irs.gov/retirement-plans/plan-participant-employee/retirement-topics-termination-of-plan",
"https://www.irs.gov/retirement-plans/plan-sponsor/payroll-deduction-ira",
"https://www.irs.gov/publications/p590a",
"https://www.dol.gov/index.php/agencies/ebsa/about-ebsa/our-activities/resource-center/publications/reporting-and-disclosure-guide-for-employee-benefit-plans",
"https://www.higginbotham.com/blog/insurance-declarations-page/",
"https://www.kff.org/health-costs/health-policy-101-employer-sponsored-health-insurance/?entry=table-of-contents-resources",
"https://www.investor.gov/additional-resources/retirement-toolkit/employer-sponsored-plans/traditional-and-roth-401k-plans",
"https://www.investor.gov/additional-resources/retirement-toolkit/employer-sponsored-plans/403b-and-457b-plans",
"https://www.investor.gov/additional-resources/retirement-toolkit/employer-sponsored-plans/employee-stock-ownership-plans",
"https://www.investor.gov/additional-resources/retirement-toolkit/self-directed-plans-individual-retirement-accounts-iras",
"https://ownyourfuture.vanguard.com/content/en/learn/saving-for-retirement/unlock-your-401k-full-potential.html",
"https://ownyourfuture.vanguard.com/content/en/learn/financial-planning/vanguards-4-principles-for-investing-success.html",
"https://ownyourfuture.vanguard.com/content/en/learn/financial-planning/where-do-i-get-medical-insurance.html",
"https://ownyourfuture.vanguard.com/content/en/learn/financial-planning/understanding-your-savings-options.html",
"https://www.irs.gov/retirement-plans/individual-retirement-arrangements-iras",
"https://www.employeefiduciary.com/blog/401k-matching-contributions",
"https://www.farther.com/resources/foundations/how-does-the-401k-employer-match-work",
"https://humaninterest.com/learn/articles/looking-in-depth-at-the-401k-employer-match/",
"https://www.irs.gov/publications/p969",
"https://www.federalregister.gov/documents/2024/04/30/2024-08985/definition-of-employer-association-health-plans"
]

pdfs = [
"https://www.dol.gov/sites/dolgov/files/EBSA/about-ebsa/our-activities/resource-center/publications/reporting-annual-disclosure.pdf",
"https://www.irs.gov/pub/irs-pdf/p1.pdf",
"https://www.irs.gov/pub/irs-pdf/p15a.pdf",
"https://www.irs.gov/pub/irs-pdf/p525.pdf",
"https://www.irs.gov/pub/irs-pdf/p571.pdf",
"https://www.irs.gov/pub/irs-pdf/p575.pdf",
"https://www.irs.gov/pub/irs-pdf/p590a.pdf",
"https://www.irs.gov/pub/irs-pdf/p590b.pdf",
"https://www.irs.gov/pub/default_path_no_value/p5875.pdf",
"https://www.cms.gov/files/document/nsa-health-insurance-basics.pdf",
"https://www.dol.gov/sites/dolgov/files/ebsa/about-ebsa/our-activities/resource-center/publications/maximize-savings-retirement-plan-investment-tips.pdf",
"https://www.dol.gov/sites/dolgov/files/ebsa/about-ebsa/our-activities/resource-center/publications/401k-plan-fees.pdf",
"https://www.dol.gov/sites/dolgov/files/ebsa/about-ebsa/our-activities/resource-center/fact-sheets/target-date-retirement-funds-2010.pdf",
 "https://www.cms.gov/marketplace/technical-assistance-resources/summary-of-benefits-fast-facts.pdf"
]



In [104]:
def load_html_links(urls: List[str]) -> List[Document]:
    docs: List[Document] = []
    for u in urls:
        try:
            docs.extend(WebBaseLoader([u]).load())
        except Exception as e:
            print(f"[HTML skip] {u} -> {e}")
    return docs

def load_pdf_links(pdfs: List[str]) -> List[Document]:
    docs: List[Document] = []
    for u in pdfs:
        try:
            docs.extend(OnlinePDFLoader(u).load())
        except Exception as e:
            print(f"[PDF skip]  {u} -> {e}")
    return docs

def dedupe_by_source(docs: List[Document]) -> List[Document]:
    seen, out = set(), []
    for d in docs:
        key = d.metadata.get("source") or hash(d.page_content)
        if key in seen:
            continue
        seen.add(key)
        out.append(d)
    return out

html_docs = load_html_links(urls)
pdf_docs  = load_pdf_links(pdfs)

documents = dedupe_by_source(html_docs + pdf_docs)
print(f"Loaded {len(documents)} unique documents "
      f"({len(html_docs)} HTML + {len(pdf_docs)} PDF before de-dup).")

Loaded 49 unique documents (35 HTML + 14 PDF before de-dup).


In [105]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100, separators = ['\n\n', '\n',''])
chunks = text_splitter.split_documents(documents)

In [106]:
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"
HF_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
client = InferenceClient(model=MODEL_ID, token=HF_TOKEN)

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma(embedding_function=embedding, persist_directory="chroma_db")
retriever = vectordb.as_retriever(search_kwargs={"k": 4})

In [107]:
class HelpType(str, Enum):
    investment = "investment"
    employer_retirement = "employer_retirement"

class EmploymentTenure(str, Enum):
    lt_3mo = "Less than three months"
    mo3_to_12 = "3-12 months"
    yr1_to_3 = "1-3 years"
    gt_3yr = "More than 3 years"

class MaritalStatus(str, Enum):
    single = "Single"
    married = "Married"
    separated = "Separated"
    divorced = "Divorced"
    widowed = "Widowed"

class RiskTolerance(str, Enum):
    low = "low"
    medium = "medium"
    high = "high"

class Concern(str, Enum):
    benefits = "Understanding my company's benefits (insurance, 401k)"
    first_paycheck = "How to manage my first real paycheck"
    debt = "Paying off student loans/debt"
    emergency_fund = "Starting an emergency fund"
    big_purchase = "Saving for a big purchase (car, down payment)"
    start_investing = "Starting to invest for the future"
    budgeting = "Budgeting and tracking my spending"
    taxes = "Understanding taxes and withholdings"

class PlanOption(str, Enum):
    k401 = "401k"
    roth_401k = "roth_401k"
    hsa = "hsa"
    fsa = "fsa"
    other = "other"

# 2-letter state code (uppercase). If you want full list, replace with Literal[...] of all states.
USStateCode = Annotated[
    str,
    StringConstraints(
        pattern=r"^[A-Z]{2}$",
        strip_whitespace=True,
        to_upper=True,
        min_length=2,
        max_length=2,
    ),
]

In [108]:
class QuizAnswers(BaseModel):
    name: str = Field(..., description="User's first or preferred name.")
    age: conint(ge=16, le=100)
    employment_tenure: EmploymentTenure
    state: USStateCode = Field(..., description="Two-letter state code, e.g., 'NC'")
    salary: confloat(ge=0) = Field(..., description="Annual gross salary in USD.")
    marital_status: MaritalStatus
    help_type: HelpType
    top_concerns: List[Concern] = Field(default_factory=list)
    risk_tolerance: Optional[RiskTolerance] = None

class BenefitsDetails(BaseModel):
    employer_offers_health_insurance: Optional[bool] = None
    enrolled_in_health_insurance: Optional[Literal["Yes, I'm enrolled", "No, not yet"]] = None
    monthly_premium: Optional[confloat(ge=0)] = None
    employer_contributes_hsa_or_fsa: Optional[bool] = None
    employer_offers_retirement_plan: Optional[bool] = None
    employer_plan_options: List[PlanOption] = Field(default_factory=list)
    employer_match_percent: Optional[confloat(ge=0, le=100)] = None
    contributing_now: Optional[bool] = None
    current_monthly_contribution: Optional[confloat(ge=0)] = None

class SavingsGoalDetails(BaseModel):
    what_are_you_saving_for: Optional[str] = None
    target_amount: Optional[confloat(ge=0)] = None
    timeline_months: Optional[conint(ge=1, le=600)] = None

class HouseholdCashflow(BaseModel):
    monthly_take_home_pay: Optional[confloat(ge=0)] = None
    monthly_expenses: Optional[confloat(ge=0)] = None
    current_savings: Optional[confloat(ge=0)] = None
    notes: Optional[str] = None

class FormAnswers(BaseModel):
    cashflow: HouseholdCashflow = HouseholdCashflow()
    benefits: BenefitsDetails = BenefitsDetails()
    savings_goal: SavingsGoalDetails = SavingsGoalDetails()

class UserProfile(BaseModel):
    quiz: QuizAnswers
    form: FormAnswers

    @property
    def name(self) -> str: return self.quiz.name
    @property
    def age(self) -> int: return self.quiz.age
    @property
    def salary(self) -> float: return float(self.quiz.salary)
    @property
    def help_type(self) -> HelpType: return self.quiz.help_type

In [109]:
def _to_bool(v):
    if v is True or v is False: return v
    if isinstance(v, str):
        s = v.strip().lower()
        if s in ("true","yes","y","1"): return True
        if s in ("false","no","n","0"): return False
    return None

EMPLOYMENT_MAP = {
    "less_than_three_months": "Less than three months",
    "3-12_months": "3-12 months",
    "1-3_years": "1-3 years",
    "more_than_3_years": "More than 3 years",
}

CONCERN_MAP = {
    "understanding_my_companys_benefits": "Understanding my company's benefits (insurance, 401k)",
    "how_to_manage_my_first_real_paycheck": "How to manage my first real paycheck",
    "paying_off_student_loans_debt": "Paying off student loans/debt",
    "starting_an_emergency_fund": "Starting an emergency fund",
    "saving_for_a_big_purchase": "Saving for a big purchase (car, down payment)",
    "starting_to_invest_for_the_future": "Starting to invest for the future",
    "budgeting_and_tracking_my_spending": "Budgeting and tracking my spending",
    "understanding_taxes_and_withholdings": "Understanding taxes and withholdings",
}


In [110]:
def build_profile_from_frontend(quizResponse: dict, formData: dict) -> UserProfile:
    # Parse numerics with grace
    def to_num(v):
        if v in (None, ""): return None
        try:
            return float(str(v).replace(",","").replace("$","").strip())
        except Exception:
            return None

    employment = EMPLOYMENT_MAP.get(
        (quizResponse.get("employment_duration") or "").strip().lower(),
        (quizResponse.get("employment_duration") or "").replace("_", " ").title()
    )
    concerns = [
        CONCERN_MAP.get(x, x.replace("_", " ")) for x in (quizResponse.get("top_financial_concerns") or [])
    ]

    quiz = {
        "name": quizResponse.get("user_name"),
        "age": int(quizResponse.get("age")),
        "employment_tenure": employment,
        "state": (quizResponse.get("state") or "").strip().upper()[:2],
        "salary": to_num(quizResponse.get("annual_salary")),
        "marital_status": quizResponse.get("marital_status"),
        "help_type": quizResponse.get("help_type", "employer_retirement"),
        "top_concerns": concerns,
        "risk_tolerance": quizResponse.get("risk_tolerance")
    }

    cashflow = {
        "monthly_take_home_pay": to_num(formData.get("monthly_take_home")),
        "monthly_expenses": to_num(formData.get("monthly_expenses")),
        "current_savings": to_num(formData.get("current_savings")),
        "notes": formData.get("additional_context") or None
    }

    benefits = {
        "employer_offers_health_insurance": _to_bool(formData.get("has_employer_health_insurance")),
        "enrolled_in_health_insurance": "Yes, I'm enrolled" if _to_bool(formData.get("enrolled_in_health_insurance")) else ("No, not yet" if _to_bool(formData.get("enrolled_in_health_insurance")) is False else None),
        "monthly_premium": to_num(formData.get("health_insurance_premium")),
        "employer_contributes_hsa_or_fsa": _to_bool(formData.get("has_hsa_fsa")),
        "employer_offers_retirement_plan": _to_bool(formData.get("has_employer_retirement")),
        "employer_plan_options": formData.get("employer_plan_options") or [],
        "employer_match_percent": to_num(formData.get("employer_match_percentage")),
        "contributing_now": _to_bool(formData.get("contributing_to_retirement")),
        "current_monthly_contribution": to_num(formData.get("retirement_contribution_amount")),
    }

    goal_months = None
    if formData.get("goal_timeline_years") not in (None, ""):
        try:
            goal_months = int(round(float(formData.get("goal_timeline_years")) * 12))
        except Exception:
            goal_months = None

    savings_goal = {
        "what_are_you_saving_for": formData.get("financial_goal_description"),
        "target_amount": to_num(formData.get("goal_target_amount")),
        "timeline_months": goal_months
    }

    return UserProfile(
        quiz=quiz,
        form={"cashflow": cashflow, "benefits": benefits, "savings_goal": savings_goal}
    )


In [111]:
def create_prompt(question: str, contexts: List[str], max_ctx_chars: int = 6000) -> str:
    joined = "\n\n".join(contexts) if contexts else ""
    context = joined[:max_ctx_chars]
    system = (
        "You are a question-answering assistant that must use ONLY the provided context.\n"
        "If the answer cannot be found in the context, reply exactly: \"I don't know.\" "
        "Do NOT make up facts. Prefer quoting or paraphrasing sentences from the context.\n"
    )
    instr = (
        "Guidelines:\n"
        "1) Keep outputs concise but clear.\n"
        "2) Cite phrases and include source tags/URLs when helpful.\n"
        "3) If the question asks for something not in the context, say: I don't know.\n"
        '4) Output valid JSON with keys: "greeting", "recommendations", "warnings", "as_of_year". '
        'Each recommendation = {"title","summary","steps","considerations","citations"}.\n'
    )
    return f"{system}\n{instr}\nQuestion:\n{question}\n\nContext:\n{context}\n\nAnswer (JSON only):"

def retrieve_context_texts(query: str, k: int = 4) -> Tuple[List[str], List[str]]:
    docs = retriever.invoke(query)
    contexts = [d.page_content for d in docs]
    sources  = [d.metadata.get("source", "") for d in docs]
    return contexts, sources

def retrieve_contexts_for_profile(p: UserProfile, k_each: int = 2, k_total: int = 8) -> Tuple[List[str], List[str]]:
    queries = []
    if p.help_type.value.startswith("employer"):
        queries += ["401(k) basics fees match", "Roth 401(k) vs Traditional 401(k)", "auto-enrollment target-date funds"]
    if any(opt.value.lower() in ("hsa","fsa") for opt in p.form.benefits.employer_plan_options):
        queries += ["HSA eligibility HDHP tax benefits vs FSA"]
    if p.form.savings_goal.what_are_you_saving_for:
        queries += ["saving plan contribution priority emergency fund rule of thumb"]

    seen, contexts, sources = set(), [], []
    for q in queries[:4] or ["retirement plan basics"]:
        ctxs, srcs = retrieve_context_texts(q, k=k_each)
        for c, s in zip(ctxs, srcs):
            key = (s, c[:120])
            if key in seen:
                continue
            seen.add(key)
            contexts.append(c)
            sources.append(s)
            if len(contexts) >= k_total: break
        if len(contexts) >= k_total: break
    return contexts, sources

def build_question_from_profile(p: UserProfile) -> str:
    opts = ", ".join([o.value for o in p.form.benefits.employer_plan_options]) or "none provided"
    concerns = ", ".join([c.value for c in p.quiz.top_concerns]) or "unspecified"
    return (
        "Create a personalized plan based ONLY on the provided context and the user profile:\n"
        f"- Name: {p.name}\n"
        f"- Age: {p.age}\n"
        f"- Salary: ${p.salary:,.0f}\n"
        f"- State: {p.quiz.state}\n"
        f"- Employment tenure: {p.quiz.employment_tenure}\n"
        f"- Marital status: {p.quiz.marital_status}\n"
        f"- Help type: {p.help_type}\n"
        f"- Top concerns: {concerns}\n"
        f"- Employer plan options: {opts}\n"
        f"- Employer match %: {p.form.benefits.employer_match_percent or 'N/A'}\n"
        f"- Contributing now: {p.form.benefits.contributing_now}\n"
        f"- Monthly take-home: ${p.form.cashflow.monthly_take_home_pay or 'N/A'}\n"
        f"- Monthly expenses: ${p.form.cashflow.monthly_expenses or 'N/A'}\n"
        f"- Current savings: ${p.form.cashflow.current_savings or 'N/A'}\n"
        f"- Savings goal: {p.form.savings_goal.what_are_you_saving_for or 'N/A'} "
        f"target ${p.form.savings_goal.target_amount or 'N/A'} in "
        f"{p.form.savings_goal.timeline_months or 'N/A'} months\n"
        "Explain options (401k/Roth/HSA if present), contribution limits, match strategy, and actionable steps. "
        "If facts are not in the context, answer: I don't know."
    )

def generate_plan_from_profile(p: UserProfile, max_new_tokens: int = 650, temperature: float = 0.2):
    contexts, sources = retrieve_contexts_for_profile(p, k_each=2, k_total=8)
    prompt = create_prompt(build_question_from_profile(p), contexts)

    messages = [
        {"role": "system", "content": "Follow the user-provided prompt exactly."},
        {"role": "user", "content": prompt},
    ]
    try:
        out = client.chat.completions.create(
            model=MODEL_ID,
            messages=messages,
            max_tokens=max_new_tokens,
            temperature=temperature,
            stop=["Question:", "Context:"]
        )
        text = out.choices[0].message.content.strip()
    except Exception:
        text = client.text_generation(
            prompt, max_new_tokens=max_new_tokens, temperature=temperature,
            return_full_text=False, stop=["Question:", "Context:", "Answer (JSON only):"]
        ).strip()

    try:
        data = json.loads(text)
    except Exception:
        data = {
            "greeting": f"Hi {p.name}, here’s your personalized plan.",
            "recommendations": [{
                "title": "Plan Overview",
                "summary": text[:1200],
                "steps": [],
                "considerations": [],
                "citations": list({s for s in sources if s})
            }],
            "warnings": [],
            "as_of_year": None
        }
    # backfill citations if empty
    for rec in data.get("recommendations", []):
        if not rec.get("citations"):
            rec["citations"] = list({s for s in sources if s})
    return data

In [112]:
sample_frontend = {
  "quizResponse": {
    "user_name": "Alex",
    "age": 26,
    "employment_duration": "3-12_months",
    "state": "nc",
    "annual_salary": 72000,
    "marital_status": "Single",
    "help_type": "employer_retirement",
    "top_financial_concerns": [
      "understanding_my_companys_benefits",
      "saving_for_a_big_purchase"
    ],
    "risk_tolerance": "medium"
  },
  "formData": {
    "monthly_take_home": 3800,
    "monthly_expenses": 2900,
    "current_savings": 4500,
    "additional_context": "Student loan starts in 3 months.",
    "has_employer_health_insurance": "true",
    "enrolled_in_health_insurance": "true",
    "health_insurance_premium": 160,
    "has_hsa_fsa": "false",
    "has_employer_retirement": "true",
    "employer_plan_options": ["401k","roth_401k"],
    "employer_match_percentage": 4,
    "contributing_to_retirement": "true",
    "retirement_contribution_amount": 250,
    "financial_goal_description": "Car down payment",
    "goal_target_amount": 8000,
    "goal_timeline_years": 1
  }
}

In [113]:
profile = build_profile_from_frontend(sample_frontend["quizResponse"], sample_frontend["formData"])
# Pydantic validation already applied inside builder (via UserProfile construction)

plan = generate_plan_from_profile(profile)
print(json.dumps(plan, indent=2))

{
  "greeting": "Hello Alex",
  "recommendations": [
    {
      "title": "Understanding 401(k) and Roth 401(k) Plans",
      "summary": "Both plans offer tax advantages, either now or in the future.",
      "steps": [
        "Consider a traditional 401(k) for tax-deferred contributions and earnings.",
        "Consider a Roth 401(k) for tax-free earnings in retirement."
      ],
      "considerations": [
        "Traditional 401(k): Contributions are tax-deferred, but earnings may be taxed in retirement.",
        "Roth 401(k): Contributions are made with after-tax dollars, but earnings are tax-free in retirement."
      ],
      "citations": [
        "Individuals who want to save for retirement may have the option to invest in a 401(k) or Roth 401(k) plan."
      ]
    },
    {
      "title": "Contribution Limits and Employer Match Strategy",
      "summary": "I don't know",
      "steps": [],
      "considerations": [],
      "citations": [
        "/tmp/tmpo7ukrg29/tmp.pdf",
    

In [114]:
import shutil
shutil.make_archive("/content/chroma_db", "gztar", "/content", "chroma_db")

'/content/chroma_db.tar.gz'